# import libs

In [ ]:
import numpy as np
import pandas as pd
from aocd import get_data
import re
import copy

# import and clean data

In [ ]:
data = get_data(day=6, year=2024)

def parse_map(input_map):
     rows = input_map.strip().split('\n')
     map_grid = [list(row) for row in rows]
     return map_grid

def find_guard(map_grid):
    directions = {'^': (-1, 0), 'v': (1, 0), '<': (0, -1), '>': (0, 1)}
    for i, row in enumerate(map_grid):
        for j, cell in enumerate(row):
            if cell in directions:
                return (i, j, cell)
    return None

def turn_right(direction):
    right_turn = {'^': '>', '>': 'v', 'v': '<', '<': '^'}
    return right_turn[direction]

def move_forward(position, direction):
    directions = {'^': (-1, 0), 'v': (1, 0), '<': (0, -1), '>': (0, 1)}
    return (position[0] + directions[direction][0], position[1] + directions[direction][1])

def is_within_bounds(position, map_grid):
    return 0 <= position[0] < len(map_grid) and 0 <= position[1] < len(map_grid[0])

def simulate_guard(input_map):
    map_grid = parse_map(input_map)
    guard_position = find_guard(map_grid)
    if not guard_position:
        return 0
    guard_row, guard_col, direction = guard_position
    visited_positions = set()
    visited_positions.add((guard_row, guard_col))

    while True:
        next_row, next_col = move_forward((guard_row, guard_col), direction)
        if not is_within_bounds((next_row, next_col), map_grid) or map_grid[next_row][next_col] == '#':
            direction = turn_right(direction)
        else:
            guard_row, guard_col = next_row, next_col
            visited_positions.add((guard_row, guard_col))
            map_grid[guard_row][guard_col] = 'X'
            if not is_within_bounds(move_forward((guard_row, guard_col), direction), map_grid):
                break
    
    return len(visited_positions)

In [ ]:
simulate_guard(data)

# test data

In [ ]:
from copy import deepcopy

p = data.split('\n')
puzzle = []

for i in p:
    puzzle.append([j for j in i])

def direction(puzzle):
    for i in range(len(puzzle)):
        for j in range(len(puzzle[0])):
            if puzzle[i][j] in '^v><':
                return puzzle[i][j]
def starting(puzzle):
    for i in range(len(puzzle)):
        for j in range(len(puzzle[0])):
            if puzzle[i][j] in '^v><':
                return (i, j)
ans = 0
start = starting(puzzle)
dir = direction(puzzle)
puzzle2 = deepcopy(puzzle)

def walkGuard(grid, start, dir):
    pos = start
    dirs = {'<': (0, -1), '>': (0, 1), '^': (-1, 0), 'v': (1, 0)}
    prevPos = start
    walking = True
    # Add 'X' until you reach a '#'.
    while walking:
        prevPos = pos
        pos = tuple(map(lambda i, j: i + j, pos, dirs[dir]))
        if (pos[0] > (len(grid) - 1)) or (pos[1] > (len(grid[0]) - 1)) or (pos[0] < 0) or (pos[1] < 0):
            return False
        if grid[pos[0]][pos[1]] == '#':
            walking = False
            if dir == '>':
                dir = 'v'
                walkGuard(grid, prevPos, dir)
            elif dir == 'v':
                dir = '<'
                walkGuard(grid, prevPos, dir)
            elif dir == '<':
                dir = '^'
                walkGuard(grid, prevPos, dir)
            elif dir == '^':
                dir = '>'
                walkGuard(grid, prevPos, dir)
        if (pos[0] > (len(grid) - 1)) or (pos[1] > (len(grid[0]) - 1)):
            return False


for i in range(len(puzzle)):
    for j in range(len(puzzle[0])):
        puzzle2[i][j] = '#'
        try:
            walkGuard(puzzle2, start, dir)
            puzzle2 = deepcopy(puzzle)
        except RecursionError:
            puzzle2 = deepcopy(puzzle)
            ans += 1


print(f'Part Two: {ans}')